In [366]:
import pandas as pd
import numpy as np
from calendar import monthrange
import random
import datetime
import bisect
from tqdm import tqdm

In [234]:
dftrain=pd.read_csv('/mnt/nvme2tb/ffp/datasets/fires_new_norm.csv',dtype={'month':np.int32,'firedate':str})
dfallf=dftrain[dftrain['fire']==1].copy()
dfallf['year']=dfallf['firedate'].str.slice(0,4)

In [97]:
#dfallfire['month1']=dfallfire['firedate'].astype(str).str.slice(4,6)
ls=list(dfallf.groupby(['month']).agg('count').reset_index()[['month','dom_dir']].itertuples(index=False, name=None))
ls1=list(dfallf.groupby(['year']).agg('count').reset_index()[['year','dom_dir']].itertuples(index=False, name=None))

In [80]:
df2.columns

Index(['firedate', 'Unnamed: 0', 'dom_dir', 'dom_vel', 'res_max', 'dir_max',
       'max_dew_temp', 'min_dew_temp', 'mean_dew_temp', 'max_temp', 'min_temp',
       'mean_temp', 'rain_7_days', 'ndvi', 'evi', 'lst_day', 'lst_night',
       'fire', 'weekday', 'month', 'x', 'y', 'pop', 'slope', 'road_dens',
       'dem', 'aspect', 'f81', 'frequency', 'corine_gr1', 'corine_gr4',
       'corine_gr5', 'corine_gr21', 'corine_gr22', 'corine_gr23',
       'corine_gr24', 'corine_gr31', 'corine_gr32', 'corine_gr33', 'band'],
      dtype='object')

In [76]:
def get_distrib(counts):
    cd = {c[0]: c[1] for c in counts}
    s = sum(cd.values())
    cdpcs = {c: cd[c] / s for c in cd}
    p = [cdpcs[pc] for pc in cdpcs]
    return p

In [100]:
mon_distrib = get_distrib(ls1)
mon_distrib

[0.04979580138998352,
 0.17754531776169663,
 0.22024790427742352,
 0.09909006233431253,
 0.07379809414630652,
 0.058823529411764705,
 0.14924410689976356,
 0.12058465286236297,
 0.05087053091638604]

In [125]:
def get_random_cell_count(dfallf, not_burned_percent):
    f = not_burned_percent
    cntrandom = int(f / (1 - f) * len(dfallf))
    return cntrandom

In [134]:
# calculate distribution in months, years
def get_random_date_distrib(dfallf):

    month_counts_ls=list(dfallf.groupby(['month']).agg('count').reset_index()[['month','dom_dir']] \
                         .itertuples(index=False, name=None))
    mon_distrib = get_distrib(month_counts_ls)
    year_counts_ls=list(dfallf.groupby(['year']).agg('count').reset_index()[['year','dom_dir']]. \
                        itertuples(index=False, name=None))
    year_distrib = get_distrib(year_counts_ls)

    return mon_distrib, year_distrib, month_counts_ls, year_counts_ls

In [414]:
def get_fires_pd(calend=True):
    if calend:
        dftrain=pd.read_csv('/mnt/nvme2tb/ffp/datasets/traindataset_new.csv',dtype={'firedate':str})
    else:
        dftrain=pd.read_csv('/mnt/nvme2tb/ffp/datasets/traindataset_new.csv',parse_dates=['firedate'])
        dftrain['firedate']=dftrain['firedate'].apply(lambda x: x.date())
    dftrain.dropna(inplace=True)
    dfallf=dftrain[dftrain['fire']==1].copy()
    if calend:
        dftrain['month']=dftrain['month'].astype(int)
        dfallf['year']=dfallf['firedate'].str.slice(0,4)
    return dfallf

In [136]:
def get_rand_from_distrib(start, stop, distrib):
    return np.random.choice(np.arange(int(start), int(stop) + 1), p=distrib)

In [137]:
def get_min_max_year_month(dfallf):
    return dfallf['year'].min(), dfallf['year'].max(), dfallf['month'].min(), dfallf['month'].max()

In [1]:
# select random dates according previously calculated distribution
def select_random_dates(cntrandom, dff):
    mdist, ydist, mcount, ycount = get_random_date_distrib(dff)
    miny, maxy, minm, maxm = get_min_max_year_month(dff)
    noburndatelist = []
    for i in range(cntrandom):
        year = get_rand_from_distrib(miny, maxy, ydist)
        month = get_rand_from_distrib(minm, maxm, mdist)
        daysofmonth = monthrange(year, month)[1]
        day = random.randrange(1, daysofmonth + 1)
        randdate = datetime.date(year=year, day=day, month=month)
        noburndatelist.append(randdate)
    return noburndatelist

In [229]:
def addxyhash(df):
    df["xyhash"]=(df["x"]*10000).astype('float').astype('int').astype('str')+ \
    (df["y"]*10000).astype('float').astype('int').astype('str')
    df.set_index("xyhash", inplace=True)

In [389]:
def validate_date(idc, burnids, burndict, randate):
    if BinarySearch(burnids, idc):
        #print(randate)
        # bl = get_sublist(burndict[idc],"<=", randate)
        bl = [e for e in burndict[idc] if e <= randate]
        #print(bl)
        # al = get_sublist(burndict[idc],">=", randate)
        bdate = max(bl) if bl else None
        #print(bdate)
        #print(randate - bdate)
        # adate = min(al) if al else None
        if (bdate and randate - bdate < datetime.timedelta(days=2 * 365.25)):
            return False
        else:
            return True
    else:
        return True

In [338]:
def get_burnid_dates(burnedrecs):
    burndict = {}
    burnids = []
    for ind, rec in burnedrecs.iterrows():
        if BinarySearch(burnids, ind):
            burndict[ind].append(rec['firedate'])
        else:
            burndict[ind] = [rec['firedate']]
            bisect.insort(burnids, ind)
    return burndict, burnids

In [339]:
def BinarySearch(a, x):
    i = bisect.bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return True
    else:
        return False

In [ ]:
def valid_proximity(idc, maskedlist):
    proxyids = [idc - 1, idc - 2228, idc - 2227, idc - 2226, idc + 1, idc + 2228, idc + 2227, idc + 2226]
    return all(BinarySearch(maskedlist, mid) for mid in proxyids)
    # return all(mid in maskedlist for mid in proxyids)

In [399]:
def select_random_cell(dateslist):
    cntrandom = len(dateslist)
    randomcells = []
    dfday = pd.read_csv('/mnt/nvme2tb/ffp/datasets/test/2019/20190930_df.csv')
    addxyhash(dfday)
 
    dffires=get_fires_pd(calend=False)
    addxyhash(dffires)
    
    burndict, burnids = get_burnid_dates(dffires)

    for i in tqdm(range(cntrandom)):
        rec = dfday.sample()
        while not validate_date(rec.index[0], burnids, burndict, dateslist[i]): #\
            #or not valid_proximity(rec['id'], maskids):
            rec = dfday.sample()
        randomcells.append({'xyhash':rec.index[0], 'firedate':dateslist[i].strftime("%Y%m%d")})
    return pd.DataFrame(randomcells)


In [415]:
#not_burned_percent = 0.667
not_burned_percent = 0.1
dff=get_fires_pd(calend=True)
cntrandom=get_random_cell_count(dff, not_burned_percent)

In [392]:
dl=select_random_dates(cntrandom, dff)

In [409]:
dffires=get_fires_pd(calend=False)
addxyhash(dffires)   

In [382]:
burndict, burnids = get_burnid_dates(dffires)

In [380]:
rec = dfday.sample()
validate_date(rec.index[0], burnids, burndict, dl[10])

True

In [410]:
dfcells=select_random_cell(dl)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1550/1550 [00:09<00:00, 163.26it/s]


In [404]:
dfcells=dfcells.set_index('xyhash')

In [ ]:
trainset=pd.DataFrame()
countd=0
for d in dates:
    countd+=1
    dcells=dfcells.loc[dfcells['firedate']==d]
    fnc=os.path.join(folder,"%s_df.nc"%d)
    if not os.path.isfile(fnc): continue
    dsday=xr.load_dataset(fnc)
    if int(d[:4])>=2015:
        dsdayall=xr.merge([dsday,static_af15],combine_attrs='drop')
    else:
        dsdayall=xr.merge([dsday,static_bef15],combine_attrs='drop')
    for ind,row in dcells.iterrows():
        xp=int(ind[0:6])/10000
        yp=int(ind[6:])/10000
        dspoint=dsdayall.loc[dict(x=slice(xp-0.0001, xp+0.0001), y=slice(yp+0.0001, yp-0.0001))]
        trainset=pd.concat([trainset,dspoint.to_dataframe().reset_index()])
    sys.stdout.write("\r days : %d/%d , rows : %d/%d"%(countd,len(dates),len(trainset), len(dfcells)))
    sys.stdout.flush()
trainset.to_csv('/media/toshiba-hdd/mlfires/dataset/classic/train_new_sample.csv', index=False)    

In [450]:
df_train_nof=pd.read_csv('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2.csv')
df_train_full=pd.read_csv('/mnt/nvme2tb/ffp/datasets/train/traindataset_new.csv')
df_train_fire=df_train_full[df_train_full['fire']==1]

In [422]:
df_train_nof.drop(columns=["time"], inplace=True)

In [423]:
df_train_nof.to_csv('/mnt/nvme2tb/ffp/datasets/train_new_sample_1_2.csv', index=False)

In [451]:
df_train_new=pd.concat([df_train_nof,df_train_fire])
df_train_new

,y,x,dom_dir,dom_vel,res_max,dir_max,max_dew_temp,min_dew_temp,mean_dew_temp,max_temp,...,corine_gr5,corine_gr21,corine_gr22,corine_gr23,corine_gr24,corine_gr31,corine_gr32,corine_gr33,band,firedate
0,40.467088,23.074756,1.0,2.498601,2.724211,8.0,292.18530,285.96875,288.26492,303.36084,...,0.0,0.234590,0.0,0.0,0.0,0.000000,0.765410,0.0,1,NaN
1,37.188885,22.136654,2.0,1.570616,3.125654,6.0,290.27320,284.60938,287.80234,303.68848,...,0.0,0.000000,0.0,0.0,0.0,0.679758,0.320242,0.0,1,NaN
2,39.369199,20.652184,2.0,1.343329,2.530426,7.0,290.66480,286.22095,288.11493,302.09400,...,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,1,NaN
3,40.905212,21.837698,1.0,1.583441,1.583441,1.0,282.92847,277.75635,278.98895,299.17260,...,0.0,0.000000,0.0,0.0,0.0,0.893161,0.106839,0.0,1,NaN
4,37.529076,22.461381,2.0,2.050971,2.050971,2.0,289.03296,279.86280,283.63412,304.46826,...,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27191,40.704190,23.296395,8.0,1.491224,1.588163,4.0,286.17700,279.71802,283.81677,295.99316,...,0.0,0.217922,0.0,0.0,0.0,0.663003,0.119075,0.0,1,20181011.0
27192,40.704190,23.301550,8.0,1.491224,1.588163,4.0,286.17700,279.71802,283.81677,295.99316,...,0.0,0.009555,0.0,0.0,0.0,0.990445,0.000000,0.0,1,20181011.0
27193,40.704190,23.306704,8.0,1.491224,1.588163,4.0,286.17700,279.71802,283.81677,295.99316,...,0.0,0.000000,0.0,0.0,0.0,1.000000,0.000000,0.0,1,20181011.0
27194,40.709345,23.296395,8.0,1.491224,1.588163,4.0,286.17700,279.71802,283.81677,295.99316,...,0.0,0.977667,0.0,0.0,0.0,0.022333,0.000000,0.0,1,20181011.0


In [452]:
df_train_new[df_train_new['fire']==1]

,y,x,dom_dir,dom_vel,res_max,dir_max,max_dew_temp,min_dew_temp,mean_dew_temp,max_temp,...,corine_gr5,corine_gr21,corine_gr22,corine_gr23,corine_gr24,corine_gr31,corine_gr32,corine_gr33,band,firedate
40,38.101215,22.471690,2.0,3.168949,3.168949,2.0,287.835449,284.592773,286.648712,297.947021,...,0.0,0.000000,0.0,0.0,0.907414,0.000815,0.091771,0.0,1,20110828.0
489,38.977464,22.208815,6.0,1.134567,2.292374,2.0,285.430664,280.516846,282.376984,303.511230,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.0,1,20170827.0
591,36.987863,26.945716,1.0,10.091726,11.126124,8.0,291.604004,289.739258,290.698242,300.148438,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.0,1,20120703.0
620,40.889749,22.229433,8.0,1.982530,3.281661,3.0,292.421631,285.063965,288.430023,306.084717,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.0,1,20150807.0
1071,36.487885,22.497462,8.0,3.929210,6.515328,7.0,296.084961,294.128906,295.458954,300.822510,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.0,1,20120728.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27191,40.704190,23.296395,8.0,1.491224,1.588163,4.0,286.177000,279.718020,283.816770,295.993160,...,0.0,0.217922,0.0,0.0,0.000000,0.663003,0.119075,0.0,1,20181011.0
27192,40.704190,23.301550,8.0,1.491224,1.588163,4.0,286.177000,279.718020,283.816770,295.993160,...,0.0,0.009555,0.0,0.0,0.000000,0.990445,0.000000,0.0,1,20181011.0
27193,40.704190,23.306704,8.0,1.491224,1.588163,4.0,286.177000,279.718020,283.816770,295.993160,...,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.0,1,20181011.0
27194,40.709345,23.296395,8.0,1.491224,1.588163,4.0,286.177000,279.718020,283.816770,295.993160,...,0.0,0.977667,0.0,0.0,0.000000,0.022333,0.000000,0.0,1,20181011.0


In [453]:
df_train_new.to_csv('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2.csv', index=False)

In [454]:
df_train=pd.read_csv('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2.csv')

In [455]:
df_train[df_train['fire']==1]

,y,x,dom_dir,dom_vel,res_max,dir_max,max_dew_temp,min_dew_temp,mean_dew_temp,max_temp,...,corine_gr5,corine_gr21,corine_gr22,corine_gr23,corine_gr24,corine_gr31,corine_gr32,corine_gr33,band,firedate
27949,38.101215,22.471690,2.0,3.168949,3.168949,2.0,287.835449,284.592773,286.648712,297.947021,...,0.0,0.000000,0.0,0.0,0.907414,0.000815,0.091771,0.0,1,20110828.0
27950,38.977464,22.208815,6.0,1.134567,2.292374,2.0,285.430664,280.516846,282.376984,303.511230,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.0,1,20170827.0
27951,36.987863,26.945716,1.0,10.091726,11.126124,8.0,291.604004,289.739258,290.698242,300.148438,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.0,1,20120703.0
27952,40.889749,22.229433,8.0,1.982530,3.281661,3.0,292.421631,285.063965,288.430023,306.084717,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.0,1,20150807.0
27953,36.487885,22.497462,8.0,3.929210,6.515328,7.0,296.084961,294.128906,295.458954,300.822510,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.0,1,20120728.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42266,40.704190,23.296395,8.0,1.491224,1.588163,4.0,286.177000,279.718020,283.816770,295.993160,...,0.0,0.217922,0.0,0.0,0.000000,0.663003,0.119075,0.0,1,20181011.0
42267,40.704190,23.301550,8.0,1.491224,1.588163,4.0,286.177000,279.718020,283.816770,295.993160,...,0.0,0.009555,0.0,0.0,0.000000,0.990445,0.000000,0.0,1,20181011.0
42268,40.704190,23.306704,8.0,1.491224,1.588163,4.0,286.177000,279.718020,283.816770,295.993160,...,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.0,1,20181011.0
42269,40.709345,23.296395,8.0,1.491224,1.588163,4.0,286.177000,279.718020,283.816770,295.993160,...,0.0,0.977667,0.0,0.0,0.000000,0.022333,0.000000,0.0,1,20181011.0


In [458]:
df_t_att=pd.read_csv('/mnt/nvme2tb/ffp/datasets/test/2019/attica/20190930_df_attica_norm.csv')

In [459]:
df_t_att

,index,y,x,dom_dir,dom_vel,res_max,dir_max,max_dew_temp,min_dew_temp,mean_dew_temp,...,corine_gr5,corine_gr21,corine_gr22,corine_gr23,corine_gr24,corine_gr31,corine_gr32,corine_gr33,band,firedate
0,0,0.501172,0.427230,6.0,0.113363,0.113363,6.0,0.809781,0.700270,0.771230,...,0.341016,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,1,20190930
1,1,0.500446,0.427230,6.0,0.113363,0.113363,6.0,0.809781,0.700270,0.771230,...,0.000000,0.0,0.0,0.000000,0.366740,0.0,0.129254,0.0,1,20190930
2,2,0.500446,0.427729,6.0,0.113363,0.113363,6.0,0.809781,0.700270,0.771230,...,0.002878,0.0,0.0,0.000000,0.502457,0.0,0.262256,0.0,1,20190930
3,3,0.500446,0.428228,6.0,0.113363,0.113363,6.0,0.809781,0.700270,0.771230,...,0.001549,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,1,20190930
4,4,0.500446,0.430224,6.0,0.113363,0.113363,6.0,0.809781,0.700270,0.771230,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,1,20190930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13781,13781,0.150527,0.391304,0.0,0.225923,0.225923,8.0,0.838714,0.855479,0.835483,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.000000,0.0,1,20190930
13782,13782,0.149801,0.390306,0.0,0.225923,0.225923,8.0,0.838714,0.855479,0.835483,...,0.000000,0.0,0.0,0.084559,0.000000,0.0,0.915441,0.0,1,20190930
13783,13783,0.149801,0.390805,0.0,0.225923,0.225923,8.0,0.838714,0.855479,0.835483,...,0.000000,0.0,0.0,0.417434,0.000000,0.0,0.582565,0.0,1,20190930
13784,13784,0.149801,0.391304,0.0,0.225923,0.225923,8.0,0.838714,0.855479,0.835483,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.000000,0.0,1,20190930
